# モジュールをインポート

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
import pickle
from sklearn.preprocessing import StandardScaler

# モデルを定義

In [3]:
from tensorflow.keras.models import load_model

In [4]:
results_dir = "./results/keras_2020_01_27_14_45"

In [5]:
model = load_model(results_dir + "/model/model.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# モデルの確認

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10, 1, 4, 1)]     0         
_________________________________________________________________
conv3d (Conv3D)              (None, 10, 1, 4, 128)     384       
_________________________________________________________________
reshape (Reshape)            (None, 10, 512)           0         
_________________________________________________________________
dense (Dense)                (None, 10, 256)           131328    
_________________________________________________________________
dropout (Dropout)            (None, 10, 256)           0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 10, 256)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 10, 512)           787968

In [7]:
# opt = Adam(lr=learning_rate)
# model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

## 学習済みモデルの重みを読み込み

In [8]:
# checkpoint_path = "./results/keras_2019_11_21/keras_model/cp.ckpt"
# model.load_weights(checkpoint_path)

# Load data stream

In [9]:
# from pylsl import StreamInlet, resolve_byprop, local_clock
# from muselsl.constants import LSL_SCAN_TIMEOUT, LSL_EEG_CHUNK
# from datetime import datetime
# LSL_EEG_CHUNK = 10
# def record_stream():
#     """
#     :return: a generator to fetch EEG data from existed stream.
#     """
#     streams = resolve_byprop("type", "EEG", timeout=LSL_SCAN_TIMEOUT)
#     if len(streams) == 0:
#         raise IOError("Can't find EEG stream.")

#     inlet = StreamInlet(streams[0], max_buflen=LSL_EEG_CHUNK)

#     while True:
#         chunk, timestamp = inlet.pull_sample(timeout=1.0)
#         if timestamp:
#             timestamp = datetime.fromtimestamp(timestamp).strftime("%A, %B %d, %Y %I:%M:%S")
#             print(timestamp, end="\r", flush=True)
# record_stream()

In [10]:
from pylsl import StreamInlet, resolve_byprop
from muselsl.constants import LSL_SCAN_TIMEOUT, LSL_EEG_CHUNK

LSL_EEG_CHUNK = 20
def record_stream():
    """
    :return: a generator to fetch EEG data from existed stream.
    """
    streams = resolve_byprop("type", "EEG", timeout=LSL_SCAN_TIMEOUT)
    if len(streams) == 0:
        raise IOError("Can't find EEG stream.")

    inlet = StreamInlet(streams[0], max_buflen=LSL_EEG_CHUNK)

    while True:
        yield inlet.pull_chunk(timeout=1.0, max_samples=LSL_EEG_CHUNK)
#         return inlet.pull_chunk(timeout=1.0, max_samples=LSL_EEG_CHUNK)


In [12]:
samples, timestamps = next(record_stream())
# print(samples)
samples = np.array(samples)
print(samples)
# for sample in samples:
#     print(sample[:4])

[[  -36.62109375   -47.36328125    54.6875       -46.38671875
    732.91015625]
 [  -63.4765625    -75.68359375    61.5234375    -77.63671875
   -616.69921875]
 [  -62.98828125   -80.078125      58.59375      -79.58984375
   -907.2265625 ]
 [  -39.55078125   -57.6171875     54.19921875   -56.640625
    116.2109375 ]
 [  -25.390625     -47.8515625     54.19921875   -41.9921875
    964.35546875]
 [  -56.15234375   -74.21875       59.08203125   -67.87109375
   -210.44921875]
 [  -68.84765625   -77.1484375     58.59375      -84.47265625
   -987.79296875]
 [  -53.7109375    -63.4765625     52.24609375   -74.70703125
   -231.4453125 ]
 [  -32.71484375   -41.9921875     44.921875     -46.875
    908.69140625]
 [  -53.22265625   -70.3125        50.29296875   -64.94140625
    167.48046875]
 [  -72.265625     -87.890625      56.640625     -90.8203125
  -1000.        ]
 [  -61.5234375    -66.40625       56.640625     -78.125
   -595.21484375]
 [  -36.1328125    -46.875         52.24609375   -54.6

In [13]:
for timestamp in timestamps:
    print(timestamp)

1580196165.288805
1580196165.2927113
1580196165.2966175
1580196165.3005238
1580196165.30443
1580196165.3083363
1580196165.3122425
1580196165.3161488
1580196165.320055
1580196165.3239613
1580196165.3278675
1580196165.3317738
1580196165.33568
1580196165.3395863
1580196165.3434925
1580196165.3473988
1580196165.351305
1580196165.3552113
1580196165.3591175
1580196165.3630238


In [14]:
# print("['TP9', 'AF7', 'AF8', 'TP10', 'Right AUX']")
# print(np.array(next(record_stream())[0])[:, 0:4])

# Preprocessing

In [15]:
with open(results_dir + "/ohe", "rb") as file:
    ohe = pickle.load(file)

In [16]:
true_labels = ohe.categories_[0]
print(true_labels)

['eye_close' 'image_open&close_both_feet' 'image_open&close_both_fists'
 'image_open&close_left_fist' 'image_open&close_right_fist']


In [17]:
with open(results_dir + "/scaler", "rb") as file:
    scaler = pickle.load(file)

In [18]:
scaler.mean_

array([ -0.3579738 , -18.1131563 , -16.02667461,  -1.57742181])

In [19]:
scaler.scale_

array([ 59.46182455, 107.01945638, 104.94636896,  62.93448588])

In [20]:
from matplotlib import animation
import time

In [21]:
def feature_normalize(data):
	mean = data[data.nonzero()].mean()
	sigma = data[data.nonzero()].std()
	data_normalized = data
	data_normalized[data_normalized.nonzero()] = (data_normalized[data_normalized.nonzero()] - mean)/sigma
	return data_normalized

In [ ]:
from IPython.display import clear_output
# print("23:51:39:746923  [[False False  True False False]]")
print(true_labels)
# print("Timestamp          Results")
for stream, timestamps in record_stream():
    stream = np.array(stream)
    stream = stream[::2]
    stream = stream[:, :4]
#     stream = (stream - scaler.mean_) / scaler.scale_
    timestamps = timestamps[::2]
    buffer_3d = []
    buffer_2d = np.zeros((1, 4))
    for data in stream:
        data = feature_normalize(data)
        
#         buffer_2d[1, 3] = data[1]
#         buffer_2d[1, 7] = data[2]
#         buffer_2d[5, 1] = data[0]
#         buffer_2d[5, 9] = data[3]
        buffer_2d[0, 0] = data[1]
        buffer_2d[0, 1] = data[2]
        buffer_2d[0, 2] = data[0]
        buffer_2d[0, 3] = data[3]
        buffer_3d.append(buffer_2d)
    buffer_3d = np.array(buffer_3d)
    X_test = buffer_3d.reshape((1, 10, 1, 4, 1))
    y_pred = model.predict(X_test, batch_size=1, verbose=0)
    y_pred_bool = np.argmax(y_pred, axis=1)
#     print(y_pred.round(3), end="\r", flush=True)
#     print(y_pred.shape)
    y_pred_confidence = (y_pred > 0.6)
#     print(y_pred_confidence.shape)
    y_pred_confidence_bool = np.argmax(y_pred_confidence, axis=1)
#     print(y_pred_confidence)
#     print(y_pred_confidence_bool)
#     if True in pred_confidence:
        
#     print(y_pred.round(3), end="\r", flush=True)
#     print(datetime.fromtimestamp(timestamps[-1]).strftime("%H:%M:%S:%f")
#           + "  " + str(y_pred), end="\r", flush=True)
#     print(datetime.fromtimestamp(timestamps[-1]).strftime("%H:%M:%S:%f")
#           + "  " + str(true_labels[int(y_pred_bool)]), end="\r", flush=True)
    print(datetime.fromtimestamp(timestamps[-1]).strftime("%H:%M:%S:%f")
          + "  " + str(y_pred_confidence), end="\r", flush=True)
#     print(y_pred.reshape(-1).shape)    
#     print(y_pred, end="\n")
#     print(flush=True)
#     y_pred_bool = np.argmax(y_pred, axis=1)
#     print(true_labels[int(y_pred_bool)], end="\r", flush=True)
#     print(true_labels[int(y_pred_bool)])

['eye_close' 'image_open&close_both_feet' 'image_open&close_both_fists'
 'image_open&close_left_fist' 'image_open&close_right_fist']
